In [2]:
import qubovert as qv
from qubovert import boolean_var

In [3]:
def latex(model, other=''):
    from IPython.display import display, Math
    text = model.pretty_str().replace('x', '').replace('z', '').replace('(', '').replace(')', '').replace('__a', 'a_') + other
    display(Math(text))
    
def latex_qubo(model):
    from IPython.display import display, Math
    text = model.pretty_str().replace('x', r'a_').replace('z', r'').replace('(', '').replace(')', '').replace('__a', 'a_')
    display(Math(text))

In [4]:
# N = 3

# # create the variables
# t = {i: boolean_var(f't_{i}') for i in range(1,N+1)}
# e = 2
# model = qv.PCBO()
# term1 = sum(t.values()) - e
# model.add_constraint_lt_zero(term1)
# print('term1:')
# latex(term1, '< 0')
# print('model')
# latex(model)
# print('qubo')
# latex_qubo(model.to_qubo())

In [5]:
from neal import SimulatedAnnealingSampler
def solve(H):
    H_qubo = H.to_qubo().Q
    sim_sampler = SimulatedAnnealingSampler()
    samples = sim_sampler.sample_qubo(H_qubo, num_reads=NUM_READS)
    return samples

In [66]:
import random
NUM_READS = 100
lr = 0.01
num_epochs = 1000

N = 50
taus = {i: boolean_var(f't_{i}') for i in range(N)}
C = [20]
F0 = sum(taus.values())


LAMB = 1
F = [F0]

v = {i: random.random() for i in range(len(F))}
print(f'initial v: {v}')
lamb = {i: LAMB for i in range(len(F))}


for epoch in range(num_epochs):
    H = qv.PCBO()
    H += sum([t * random.random() for t in list(taus.values())]) # f_0
    for i in range(len(F)):
        H += - v[i] * F[i]

    samples = solve(H)

    for i in range(len(F)):
        
        exp_val_f_i = 0
        for sample in samples:
            H_solution = H.convert_solution(sample)
            f_i_val = F[i].value(H_solution)
            exp_val_f_i += f_i_val
        exp_val_f_i = exp_val_f_i / len(samples)
        
        v[i] = v[i] + lr * (C[i] - exp_val_f_i)
        if epoch != 0 and epoch % 100 == 0:
            print(f'epoch {epoch} v[{i}]: {v[i]}, \t Expectation Value of F{i}: {exp_val_f_i} \t Constraint Error: {sum(H_solution.values()) - C[i]}')
    if epoch != 0 and epoch % 100 == 0:
        print()

H_solution

initial v: {0: 0.5493144295038563}
epoch 100 v[0]: 0.3793144295038563, 	 Expectation Value of F0: 22.0 	 Constraint error 2

epoch 200 v[0]: 0.38921442950385604, 	 Expectation Value of F0: 20.0 	 Constraint error 0

epoch 300 v[0]: 0.369214429503856, 	 Expectation Value of F0: 15.0 	 Constraint error -5

epoch 400 v[0]: 0.41941442950385593, 	 Expectation Value of F0: 13.0 	 Constraint error -7

epoch 500 v[0]: 0.40941442950385587, 	 Expectation Value of F0: 23.0 	 Constraint error 3



KeyboardInterrupt: 